In [ ]:
from blendsql import blend
from blendsql.ingredients import LLMMap, LLMJoin, LLMQA
from blendsql.models import OpenaiLLM
from blendsql.db import SQLite
from typing import List
from textwrap import dedent
import pandas as pd
import sqlite3
import os
import pickle
import csv

In [ ]:
# fewshots
california_schools_examples = [
        {
            "question": "Is this a charter school?",
            "output_type": "boolean",
            "options": ["t", "f"],
            "examples": {
                "District: Alameda County Office of Education, School: FAME Public Charter": "t",
                "District: Alameda County Office of Education, School: Alameda County Juvenile Hall/Court": "f",
            }
        },
        {
            "question": "What is the educational option type of the school?",
            "example_outputs": ['Traditional', 'Juvenile Court School', 'County Community School', 'State Special School', 'Alternative School of Choice', 'Continuation School', 'Special Education School', 'Community Day School', 'Home and Hospital', 'Opportunity School', 'Youth Authority School', 'District Special Education Consortia School'],
            "examples": {
                "District: Alameda County Office of Education, School: FAME Public Charter": "Traditional",
                "District: El Dorado County Office of Education, School: Charter Community School Home Study Academy": "County Community School",
            }
        },
        {
            "question": "Is the school funded directly under the charter funding type?",
            "output_type": "boolean",
            "options": ["t", "f"],
            "examples": {
                "District: Alameda County Office of Education, School: FAME Public Charter": "t",
                "District: Alameda County Office of Education, School: Alameda County Juvenile Hall/Court": "f",
            }
        },
        {
            "question": "Is the school at the address a magnet school?",
            "output_type": "boolean",
            "options": ["t", "f"],
            "examples": {
                "1515 Webster Street": "f",
                "755 Oak Grove Road": "t",
            }
        },
        {
            "question": "Is the address located in Contra Costa County?",
            "output_type": "boolean",
            "options": ["t", "f"],
            "examples": {
                "1515 Webster Street": "f",
                "755 Oak Grove Road": "f",
            }
        },
        {
            "question": "Is the school operate exclusively virtual?",
            "output_type": "boolean",
            "options": ["t", "f"],
            "examples": {
                "Aspire California College Preparatory Academy": "f",
                "Dunlap Leadership Academy": "t",
            }
        },
        {
            "question": "Provide the website based on the school address.",
            "examples": {
                "5328 Brann Street": "www.urbanmontessori.org",
                "39500 Dunlap Road": "www.kcusd.com",
            }
        },
        {
            "question": "Provide the city name based on the address.",
            "examples": {
                "5328 Brann Street": "Oakland",
                "755 Oak Grove Road": "Concord",
            }
        },
        {
            "question": "Provide the county name based on the address.",
            "examples": {
                "5328 Brann Street": "Alameda",
                "755 Oak Grove Road": "Contra Costa",
            }
        },
    ]

superhero_examples = [
        {
            "question": "Provide a list of super powers seperated by comma.",
            "example_outputs":['Agility', 'Accelerated Healing', 'Lantern Power Ring', 'Dimensional Awareness', 'Cold Resistance', 'Durability', 'Stealth', 'Energy Absorption', 'Flight', 'Danger Sense', 'Underwater breathing', 'Marksmanship', 'Weapons Master', 'Power Augmentation', 'Animal Attributes', 'Longevity', 'Intelligence', 'Super Strength', 'Cryokinesis', 'Telepathy', 'Energy Armor', 'Energy Blasts', 'Duplication', 'Size Changing', 'Density Control', 'Stamina', 'Astral Travel', 'Audio Control', 'Dexterity', 'Omnitrix', 'Super Speed', 'Possession', 'Animal Oriented Powers', 'Weapon-based Powers', 'Electrokinesis', 'Darkforce Manipulation', 'Death Touch', 'Teleportation', 'Enhanced Senses', 'Telekinesis', 'Energy Beams', 'Magic', 'Hyperkinesis', 'Jump', 'Clairvoyance', 'Dimensional Travel', 'Power Sense', 'Shapeshifting', 'Peak Human Condition', 'Immortality', 'Camouflage', 'Element Control', 'Phasing', 'Astral Projection', 'Electrical Transport', 'Fire Control', 'Projection', 'Summoning', 'Enhanced Memory', 'Reflexes', 'Invulnerability', 'Energy Constructs', 'Force Fields', 'Self-Sustenance', 'Anti-Gravity', 'Empathy', 'Power Nullifier', 'Radiation Control', 'Psionic Powers', 'Elasticity', 'Substance Secretion', 'Elemental Transmogrification', 'Technopath/Cyberpath', 'Photographic Reflexes', 'Seismic Power', 'Animation', 'Precognition', 'Mind Control', 'Fire Resistance', 'Power Absorption', 'Enhanced Hearing', 'Nova Force', 'Insanity', 'Hypnokinesis', 'Animal Control', 'Natural Armor', 'Intangibility', 'Enhanced Sight', 'Molecular Manipulation', 'Heat Generation', 'Adaptation', 'Gliding', 'Power Suit', 'Mind Blast', 'Probability Manipulation', 'Gravity Control', 'Regeneration', 'Light Control', 'Echolocation', 'Levitation', 'Toxin and Disease Control', 'Banish', 'Energy Manipulation', 'Heat Resistance', 'Natural Weapons', 'Time Travel', 'Enhanced Smell', 'Illusions', 'Thirstokinesis', 'Hair Manipulation', 'Illumination', 'Omnipotent', 'Cloaking', 'Changing Armor', 'Power Cosmic', 'Biokinesis', 'Water Control', 'Radiation Immunity', 'Vision - Telescopic', 'Toxin and Disease Resistance', 'Spatial Awareness', 'Energy Resistance', 'Telepathy Resistance', 'Molecular Combustion', 'Omnilingualism', 'Portal Creation', 'Magnetism', 'Mind Control Resistance', 'Plant Control', 'Sonar', 'Sonic Scream', 'Time Manipulation', 'Enhanced Touch', 'Magic Resistance', 'Invisibility', 'Sub-Mariner', 'Radiation Absorption', 'Intuitive aptitude', 'Vision - Microscopic', 'Melting', 'Wind Control', 'Super Breath', 'Wallcrawling', 'Vision - Night', 'Vision - Infrared', 'Grim Reaping', 'Matter Absorption', 'The Force', 'Resurrection', 'Terrakinesis', 'Vision - Heat', 'Vitakinesis', 'Radar Sense', 'Qwardian Power Ring', 'Weather Control', 'Vision - X-Ray', 'Vision - Thermal', 'Web Creation', 'Reality Warping', 'Odin Force', 'Symbiote Costume', 'Speed Force', 'Phoenix Force', 'Molecular Dissipation', 'Vision - Cryo', 'Omnipresent', 'Omniscient'],
            "examples": {
                "Abe Sapien": "Accelerated Healing,Intelligence,Super Strength,Stamina,Super Speed,Invulnerability,Animation,Super Breath",
                "Abin Sur": "Lantern Power Ring",
            }
        },
        {
            "question": "Does the hero has Super Strength?",
            "output_type": "boolean",
            "options": ["t", "f"],
            "examples": {
                "Abin Sur": "f",
                "Abomination": "t",
            }
        },
        {
            "question": "Does the hero has blue eye?",
            "output_type": "boolean",
            "options": ["t", "f"],
            "examples": {
                "Abe Sapien": "t",
                "Lucifer Morningstar": "f",
            }
        },
        {
            "question": "What is the colour of 3-D Man skin?",
            "example_outputs":['No Colour', 'Amber', 'Auburn', 'Black', 'Black/Blue', 'Blond', 'Blue', 'Blue/White', 'Brown', 'Brown/Black', 'Brown/White', 'Gold', 'Grey', 'Green', 'Green/Blue', 'Hazel', 'Indigo', 'Magenta', 'Orange', 'Orange/White', 'Pink', 'Purple', 'Red', 'Red/Black', 'Red/Grey', 'Red/Orange', 'Red/White', 'Silver', 'Strawberry Blond', 'Violet', 'White', 'White/Red', 'Yellow', 'Yellow/Blue', 'Yellow/Red'],
            "examples": {
                "3-D Man": "No Colour",
            }
        },
        {
            "question": "Is the publisher Marvel Comics?",
            "output_type": "boolean",
            "options": ["t", "f"],
            "examples": {
                "3-D Man": "t",
                "Abe Sapien": "f",
            }
        },
        {
            "question": "What is the race of the hero?",
            "example_outputs":['-', 'Alien', 'Alpha', 'Amazon', 'Android', 'Animal', 'Asgardian', 'Atlantean', 'Bizarro', 'Bolovaxian', 'Clone', 'Cosmic Entity', 'Cyborg', 'Czarnian', 'Dathomirian Zabrak', 'Demi-God', 'Demon', 'Eternal', 'Flora Colossus', 'Frost Giant', 'God / Eternal', 'Gorilla', 'Gungan', 'Human', 'Human / Altered', 'Human / Clone', 'Human / Cosmic', 'Human / Radiation', 'Human-Kree', 'Human-Spartoi', 'Human-Vulcan', 'Human-Vuldarian', 'Icthyo Sapien', 'Inhuman', 'Kakarantharaian', 'Korugaran', 'Kryptonian', 'Luphomoid', 'Maiar', 'Martian', 'Metahuman', 'Mutant', 'Mutant / Clone', 'New God', 'Neyaphem', 'Parademon', 'Planet', 'Rodian', 'Saiyan', 'Spartoi', 'Strontian', 'Symbiote', 'Talokite', 'Tamaranean', 'Ungaran', 'Vampire', 'Xenomorph XX121', 'Yautja', "Yoda's species", 'Zen-Whoberian', 'Zombie'],
            "examples": {
                "3-D Man": "-",
                "Lucifer Morningstar": "God / Eternal",
            }
        },
        {
            "question": "Is the hero male?",
            "output_type": "boolean",
            "options": ["t", "f"],
            "examples": {
                "Abe Sapien": "t",
                "Abomination": "t",
            }
        },
        {
            "question": "Provide the moral_alignment.",
            "examples": {
                "Abe Sapien": "Good",
                "Abomination": "Bad",
            }
        },
    ]

formula_1_examples = [
        {
            "question": "Provide the country name.",
            "examples": {
                "Kuala Lumpur": "Malaysia",
                "Sakhir": "Bahrain",
            }
        },
        {
            "question": "Provide the wiki url.",
            "examples": {
                "Istanbul, Istanbul Park": "http://en.wikipedia.org/wiki/Istanbul_Park",
                "Monte-Carlo, Circuit de Monaco": "http://en.wikipedia.org/wiki/Circuit_de_Monaco",
                "Malaysian Grand Prix 2007": "http://en.wikipedia.org/wiki/2007_Formula_One_season"
            }
        },
        {
            "question": "Provide the longitude based on location (float).",
            "examples": {
                "Kuala Lumpur": "101.738",
                "Sakhir": "50.5106",
            }
        },
        {
            "question": "Provide the latitude based on location (float).",
            "examples": {
                "Kuala Lumpur": "2.76083",
                "Sakhir": "26.0325",
            }
        },
        {
            "question": "Provide the nationality.",
            "examples": {
                "BMW Sauber": "German",
                "LewisHamilton": "Britis",
            }
        },
        {
            "question": "Provide the race date.",
            "examples": {
                "Australian Grand Prix 2009": "2009-03-29",
                "Bahrain Grand Prix 2008": "2008-04-06",
            }
        },
        {
            "question": "Provide the date of birth.",
            "examples": {
                "NickHeidfeld": "1977-05-10",
                "LewisHamilton": "1985-01-07",
            }
        },
    ]
european_football_2_examples = [
        {
            "question": "Provide the football league name.",
            "examples": {
                "KRC Genk vs Beerschot AC": "Belgium Jupiler League",
                "Manchester United, Newcastle United": "England Premier League",
            }
        },
        {
            "question": "Provide the wiki url.",
            "examples": {
                "Istanbul, Istanbul Park": "http://en.wikipedia.org/wiki/Istanbul_Park",
                "Monte-Carlo, Circuit de Monaco": "http://en.wikipedia.org/wiki/Circuit_de_Monaco",
                "Malaysian Grand Prix 2007": "http://en.wikipedia.org/wiki/2007_Formula_One_season"
            }
        },
        {
            "question": "Provide the player height.",
            "examples": {
                "Aaron Appindangoye": "182.88",
                "Aaron Doran": "170.18",
            }
        },
        {
            "question": "Provide the total goals for this match.",
            "examples": {
                "SV Zulte-Waregem vs Sporting Lokeren on 2008-08-16": "0",
                "FCV Dender EH vs Standard de Liège on 2008-08-16": "4",
            }
        },
        {
            "question": "Is the player’s preferred foot left?",
            "examples": {
                "Aaron Hughes": "f",
                "Aaron Hunt": "t",
            }
        },
        {
            "question": "Provide the 3 letters short team name.",
            "examples": {
                "KRC Genk": "GEN",
                "Beerschot AC": "BAC",
            }
        },
        {
            "question": "Provide the player’s birthday (YYYY-MM-DD).",
            "examples": {
                "Aaron Hughes": "1979-11-08",
                "Aaron Hunt": "1986-09-04",
            }
        },
    ]

In [ ]:
OpenaiLLM( model, env="./../../../.blendsql_env", config={"temperature": 0}).cache.clear()

In [ ]:
fewshot = 5
LLMMap.from_args(
    few_shot_examples= california_schools_examples+superhero_examples,
    # Will fetch `k` most relevant few-shot examples using DPR retriever
    k=fewshot
),

LLMQA.from_args(
    few_shot_examples= [
        {
            "question": "Provide a list of super powers seperated by comma.",
            "context": pd.DataFrame(
                data=[["Abomination"]], columns=["hero"]
            ),
            "answer": "Accelerated Healing,Intelligence,Super Strength,Stamina,Super Speed,Invulnerability,Animation,Super Breath",
        }, 
        {
            "question": "What is the skin colour?",
            "context": pd.DataFrame(
                data=[["Abin Sur"]], columns=["hero"]
            ),
            "answer": "Red",
        },
        {
            "question": "Provide the 3 letters short team name.",
            "context": pd.DataFrame(
                data=[["Sporting Lokeren"]], columns=["long team name"]
            ),
            "answer": "LOK",
        },
        {
            "question": "What is the player birthday (format: YYYY-MM-DD HH:MI:SS)?",
            "context": pd.DataFrame(
                data=[["Aaron Appindangoye", 187]], columns=["player name", "weight"]
            ),
            "answer": "1992-02-29",
        },
        {
            "question": "Which country is the league from?",
            "context": pd.DataFrame(
                data=[["Manchester United"]], columns=["league name"]
            ),
            "answer": "'England",
        },
        {
            "question": "Which is the driver code",
            "context": pd.DataFrame(
                data=[["LewisHamilton"]], columns=["driver name"]
            ),
            "answer": "HAM",
        }
        
    ],
    # Will fetch `k` most relevant few-shot examples using DPR retriever
    k=fewshot
),

def blendsql_execution(predicted_query, db, blender):
    ans = blend(
        query=predicted_query,
        db=db,
        ingredients={LLMMap, LLMQA, LLMJoin},
        verbose=False,
        default_model=blender
    )
    return ans

In [ ]:
def execute(sql, database):
    db_path = ''
    db_path = f'./../databases/dev_databases/{database}/{database}.sqlite'
        
    if not os.path.isfile(db_path):
        print("cannot find file")
        return False
    results = ''
    connection = sqlite3.connect(db_path, uri=True)
    cursor = connection.cursor()
    try:
        results = cursor.execute(sql).fetchall()
    except Exception as ex:
        cursor.close()
        connection.close()
        print(ex)
        return False

    cursor.close()
    connection.close()
    return results

def evalfunc(source_results, gold_sql, gold_results):    
    if len(source_results) != len(gold_results):
            return False
    if 'ORDER BY' in gold_sql:
        for a, b in zip(source_results, gold_results):
            lhs = tuple(sorted(list(a), key=lambda x: hash(x)))
            rhs = tuple(sorted(list(b), key=lambda x: hash(x)))
            if lhs != rhs:
                return False
    else:
        lset, rset = set(), set()
        for a, b in zip(source_results, gold_results):
            lset.add(tuple(sorted(list(a), key=lambda x: hash(x))))
            rset.add(tuple(sorted(list(b), key=lambda x: hash(x))))
        if lset != rset:
            return False
    return True

In [ ]:
model = "gpt-3.5-turbo"


for fname in ['european_football_2_HybridQueries.csv', 'CASchool_HybridQueries.csv', 'formula_1_HybridQueries.csv', 'superhero_HybridQueries.csv']:
    logs = {}
    gold_sqls = []
    hybrid_sqls = []
    dbs = []
    with open(f'./../beyond-database-questions/{fname}', mode='r') as file:
        reader = csv.reader(file)
        for row in reader:
            gold_sqls.append(row[3])
            hybrid_sqls.append(row[5])
            dbs.append(row[0])
    execution_matches = 0
    for db, gold, hybridsql in zip(dbs, gold_sqls, hybrid_sqls):
        gold_output = execute(gold, db)
        db_path = f'./../databases/dev_databases/{db}/{db}.sqlite'
        hybrid_output = blendsql_execution(hybridsql, SQLite(db_path), OpenaiLLM(
            model,
            env="./../../../.blendsql_env",
            config={"temperature": 0}))
        logs[hybridsql] = hybrid_output
        tuple_list = list(hybrid_output.df.to_records(index=False))
        if evalfunc(tuple_list, gold, gold_output):
            print('execution match')
            execution_matches += 1

    dbname = dbs[0]
    with open(f'hybridquery_{model}_{dbname}_{fewshot}shot.pkl', 'wb') as file:  # 'wb' means write in binary mode
        pickle.dump(logs, file)
    print(fname, execution_matches, execution_matches/len(gold_sqls))

In [ ]:
log = None
fname = ''
with open(fname, 'rb') as file:  # 'rb' means read in binary mode
    log = pickle.load(file)